In [1]:
import sys
import pathlib

%load_ext lab_black
sys.path.append(r"src")
sys.path.append(r"../ipyautoui/src")
sys.path.append(r"../ipyrun/src")

from ipyrun import RunApp, BatchApp, ConfigShell, DefaultConfigShell, RunShellActions
from ipyrun.runshell import (
    AutoDisplayDefinition,
    create_autodisplay_map,
    ConfigBatch,
    DefaultConfigShell,
    DefaultBatchActions,
    BatchShellActions,
)
from ipyautoui._utils import load_PyObj
from ipyautoui.displayfile_renderers import display_python_file
from ipyautoui import AutoUi
from ipyautoui import AutoDisplay

from project_tracker import *
from input_schema_project import Project

In [2]:
DIR_CORE = pathlib.Path("./src").resolve()
FPTH_SCRIPT = DIR_CORE / "script_project_summary.py"
FDIR_TEMPLATES = DIR_CORE / "templates"
NAME_JINJA_PROJECT = "project.jinja"
FPTH_JINJA_PROJECT = FDIR_TEMPLATES / NAME_JINJA_PROJECT  # FDIR_TEMPLATES /
FPTH_INPUTS_SCHEMA = DIR_CORE / "input_schema_project.py"
DIR_APPDATA = pathlib.Path("./appdata").resolve()

In [8]:
input_defn = AutoDisplayDefinition(
    path=FPTH_INPUTS_SCHEMA, obj_name="Project", ftype="in", ext=".project.json"
)
input_renderer = create_autodisplay_map(input_defn)

aui = AutoDisplay.from_paths(
    paths=[
        "/mnt/c/engDev/git_mf/project_tracker/appdata/AecTemplater/in-test.project.json"
    ],
    file_renderers=input_renderer,
)
aui

In [9]:
from ipyautoui._utils import PyObj, load_PyObj
from ipyautoui.custom.fileupload import FileUploadToDir
from input_schema_project import Project

In [10]:
config = DefaultConfigShell(
    fpth_script=FPTH_SCRIPT,
    fdir_appdata=DIR_APPDATA,
    autodisplay_definitions=[input_defn],
)
# display(config.dict())

In [11]:
from pydantic import BaseModel, validator, Field
import stringcase
import functools
from IPython.display import display, clear_output
from ipyrun.ui_add import AddNamedRun


class ProjectTrackerConfigShell(DefaultConfigShell):
    @validator("fpth_script", always=True, pre=True)
    def _set_fpth_script(cls, v, values):
        return FPTH_SCRIPT

    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        fdir = values["fdir_appdata"]
        key = values["key"]
        paths = [
            fdir / ("out-" + key + ".md"),
            fdir / ("out-" + key + ".pdf"),
            fdir / ("out-" + key + ".docx"),
        ]
        return paths

    @validator("autodisplay_definitions", always=True)
    def _autodisplay_definitions(cls, v, values):
        return [
            AutoDisplayDefinition(
                path=FPTH_INPUTS_SCHEMA,
                obj_name="Project",
                ftype="in",
                ext=".project.json",
            )
        ]

    @validator("autodisplay_inputs_kwargs", always=True)
    def _autodisplay_inputs_kwargs(cls, v, values):
        return dict(patterns="*")

    # @validator("autodisplay_outputs_kwargs", always=True)
    # def _autodisplay_outputs_kwargs(cls, v, values):
    #     return dict(patterns="*.pdf")

    @validator("long_name", always=True)
    def _long_name(cls, v, values):
        if v is None:
            return stringcase.titlecase(values["name"])
        else:
            return v

    @validator("key", always=True)
    def _key(cls, v, values):
        if v is None:
            return values["name"]
        else:
            return v

    # @validator("params", always=True)
    # def _params(cls, v, values):
    #     return {
    #         "FDIR_TEMPLATES": FDIR_TEMPLATES,
    #         "NAME_JINJA_PROJECT": NAME_JINJA_PROJECT,
    #     }


config = ProjectTrackerConfigShell(
    fpth_script=FPTH_SCRIPT,
    fdir_appdata=DIR_APPDATA,
    autodisplay_definitions=[input_defn],
    # params={"FDIR_TEMPLATES": FDIR_TEMPLATES, "NAME_JINJA_PROJECT": NAME_JINJA_PROJECT},
)
# config.dict()  # ["shell"]

In [12]:
from ipyrun.runshell import RunShellActions


def show(app):
    app.help_ui.value = False
    app.help_run.value = False
    app.help_config.value = False
    app.inputs.value = True
    app.outputs.value = False
    app.runlog.value = False


class ProjectTrackerActions(RunShellActions):
    @validator("activate", always=True)
    def _activate(cls, v, values):
        return functools.partial(show, values["app"])


config_batch = ConfigBatch(
    fdir_root=DIR_APPDATA,
    cls_config=ProjectTrackerConfigShell,
    cls_actions=ProjectTrackerActions,
    title="""# Project Tracker""",
)
# from devtools import debug

# debug(config_batch)
# print("---")

In [13]:
def fn_add_show(app):
    clear_output()
    display(AddNamedRun(fn_add=app.actions.add))


class ProjectTrackerConfigBatch(ConfigBatch):
    @validator("cls_actions", always=True)
    def _cls_actions(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        return ProjectTrackerActions

    @validator("cls_config", always=True)
    def _cls_config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        return ProjectTrackerConfigShell


class ProjectTrackerBatchActions(BatchShellActions):
    @validator("add_show", always=True)
    def _add_show(cls, v, values):
        return functools.partial(fn_add_show, values["app"])

    @validator("config", always=True)
    def _config(cls, v, values):
        """bundles RunApp up as a single argument callable"""
        if type(v) == dict:
            v = ProjectTrackerConfigBatch(**v)
        return v

    @validator("runlog_show", always=True)
    def _runlog_show(cls, v, values):
        return None


config_batch = ConfigBatch(
    fdir_root=DIR_APPDATA,
    cls_config=ProjectTrackerConfigShell,
    cls_actions=ProjectTrackerActions,
    title="""# Project Tracker""",
)
if config_batch.fpth_config.is_file():
    config_batch = ProjectTrackerConfigBatch.parse_file(config_batch.fpth_config)
app = BatchApp(config_batch, cls_actions=ProjectTrackerBatchActions)
display(app)

BatchApp(children=(VBox(children=(HTML(value='<h1>Project Tracker</h1>'), HBox(children=(Checkbox(value=False,…

In [13]:
model = Project
path = "/mnt/c/engDev/git_mf/project_tracker/appdata/AecTemplater/in-AecTemplater.project.json"

import json


def parse_json_file(path: pathlib.Path, model=None):
    """read json from file"""
    p = pathlib.Path(path)
    if model is not None:
        return model.parse_file(p).dict()
    else:
        return json.loads(p.read_text())


parse_json_file(path)

{'project_name': 'fuck',
 'project_summary': '',
 'current_status': 'active',
 'linked_files': {},
 'brief': {'client_originator': '',
  'work_type': '',
  'related_projects': '',
  'brief': '',
  'background': '',
  'value_to_practice': '',
  'description_of_output': ''},
 'resource': {'development_notes': '', 'maintainence_notes': ''},
 'programme': {'start': None,
  'draft': None,
  'beta_release': None,
  'sign_off': None,
  'distribution': None},
 'management': {'consultees': '',
  'precedents': '',
  'issues_challenges': '',
  'dissemination': ''}}

In [19]:
# BatchShellActions.load

In [15]:
app.runs.items["AecTemplater"].actions.update_status()

In [9]:
# import plotly.graph_objects as go

# name = "tracker"

# top_labels = [
#     "Strongly<br>agree",
#     "Agree",
#     "Neutral",
#     "Disagree",
#     "Strongly<br>disagree",
# ]

# colors = [
#     "rgba(38, 24, 74, 0.8)",
#     "rgba(71, 58, 131, 0.8)",
#     "rgba(122, 120, 168, 0.8)",
#     "rgba(164, 163, 204, 0.85)",
#     "rgba(190, 192, 213, 1)",
# ]

# x_data = [
#     [21, 30, 21, 16, 12],
# ]

# y_data = [
#     name,
# ]

# fig = go.Figure()

# for i in range(0, len(x_data[0])):
#     for xd, yd in zip(x_data, y_data):
#         fig.add_trace(
#             go.Bar(
#                 x=[xd[i]],
#                 y=[yd],
#                 orientation="h",
#                 marker=dict(
#                     color=colors[i], line=dict(color="rgb(248, 248, 249)", width=1)
#                 ),
#             )
#         )

# fig.update_layout(
#     xaxis=dict(
#         showgrid=False,
#         showline=False,
#         showticklabels=False,
#         zeroline=False,
#         domain=[0.15, 1],
#     ),
#     yaxis=dict(showgrid=False, showline=False, showticklabels=False, zeroline=False,),
#     barmode="stack",
#     paper_bgcolor="rgb(248, 248, 255)",
#     plot_bgcolor="rgb(248, 248, 255)",
#     margin=dict(l=120, r=10, t=140, b=80),
#     showlegend=False,
# )

# annotations = []

# for yd, xd in zip(y_data, x_data):
#     # labeling the y-axis
#     annotations.append(
#         dict(
#             xref="paper",
#             yref="y",
#             x=0.14,
#             y=yd,
#             xanchor="right",
#             text=str(yd),
#             font=dict(family="Arial", size=14, color="rgb(67, 67, 67)"),
#             showarrow=False,
#             align="right",
#         )
#     )
#     # labeling the first percentage of each bar (x_axis)
#     annotations.append(
#         dict(
#             xref="x",
#             yref="y",
#             x=xd[0] / 2,
#             y=yd,
#             text=str(xd[0]) + "%",
#             font=dict(family="Arial", size=14, color="rgb(248, 248, 255)"),
#             showarrow=False,
#         )
#     )
#     # labeling the first Likert scale (on the top)
#     if yd == y_data[-1]:
#         annotations.append(
#             dict(
#                 xref="x",
#                 yref="paper",
#                 x=xd[0] / 2,
#                 y=1.1,
#                 text=top_labels[0],
#                 font=dict(family="Arial", size=14, color="rgb(67, 67, 67)"),
#                 showarrow=False,
#             )
#         )
#     space = xd[0]
#     for i in range(1, len(xd)):
#         # labeling the rest of percentages for each bar (x_axis)
#         annotations.append(
#             dict(
#                 xref="x",
#                 yref="y",
#                 x=space + (xd[i] / 2),
#                 y=yd,
#                 text=str(xd[i]) + "%",
#                 font=dict(family="Arial", size=14, color="rgb(248, 248, 255)"),
#                 showarrow=False,
#             )
#         )
#         # labeling the Likert scale
#         if yd == y_data[-1]:
#             annotations.append(
#                 dict(
#                     xref="x",
#                     yref="paper",
#                     x=space + (xd[i] / 2),
#                     y=1.1,
#                     text=top_labels[i],
#                     font=dict(family="Arial", size=14, color="rgb(67, 67, 67)"),
#                     showarrow=False,
#                 )
#             )
#         space += xd[i]

# fig.update_layout(annotations=annotations)
# fig.add_vline(x=21, line_width=3, line_dash="dash", line_color="green")
# fig.show()